In [1]:
import albumentations as A
import cv2
import os
import numpy as np
from concurrent.futures import ThreadPoolExecutor
from tqdm import tqdm

In [2]:
# Augmentations use paraller processing,,,

transform = A.Compose([
    A.HorizontalFlip(p=0.5),
    A.RandomBrightnessContrast(p=0.2),
    A.Rotate(limit=30, p=0.5),
    A.GaussianBlur(blur_limit=3, p=0.2),
], bbox_params=A.BboxParams(format="yolo", label_fields=["class_labels"]))


In [3]:
def load_yolo_labels(label_path):
    if not os.path.exists(label_path):
        return []
    with open(label_path, "r") as f:
        return [list(map(float, line.strip().split())) for line in f.readlines()]

In [4]:
def save_yolo_labels(label_path, labels):
    with open(label_path, "w") as f:
        for label in labels:
            f.write(" ".join(map(str, label)) + "\n")

In [5]:
def augment_image(image_path, label_path, output_dir):
    image = cv2.imread(image_path)
    if image is None:
        return


In [6]:
def load_yolo_labels(label_path):
    if not os.path.exists(label_path):
        return []
    with open(label_path, "r") as f:
        return [list(map(float, line.strip().split())) for line in f.readlines()]

In [7]:
def save_yolo_labels(label_path, labels):
    with open(label_path, "w") as f:
        for label in labels:
            f.write(" ".join(map(str, label)) + "\n")
            

In [11]:
def augment_image(image_path, label_path, output_dir):
    image = cv2.imread(image_path)
    if image is None:
        return

    h, w = image.shape[:2]
    labels = load_yolo_labels(label_path)
    if not labels:
        return

    bboxes = []
    class_labels = []
    for cls, x, y, bw, bh in labels:
        x_min = x - bw / 2
        y_min = y - bh / 2
        x_max = x + bw / 2
        y_max = y + bh / 2

        if x_min < 0: 
            x_min = 0.0
        if y_min < 0: 
            y_min = 0.0
        if x_max > 1:
            x_max = 1.0
        if y_max > 1:
            y_max = 1.0

        if x_max <= x_min or y_max <= y_min:
            print(f"Skipping invalid bbox: {[x_min, y_min, x_max, y_max]}")
            continue

        bboxes.append([x_min, y_min, x_max, y_max])
        class_labels.append(cls)

    if not bboxes:
        return

    transform = A.Compose([
        A.HorizontalFlip(p=0.5),
        A.RandomBrightnessContrast(p=0.2),
        A.Rotate(limit=10, p=0.5)
    ], bbox_params=A.BboxParams(format='pascal_voc', label_fields=['class_labels']))

    os.makedirs(os.path.join(output_dir, "images"), exist_ok=True)
    os.makedirs(os.path.join(output_dir, "labels"), exist_ok=True)

    for i in range(3):
        augmented = transform(image=image, bboxes=bboxes, class_labels=class_labels)
        aug_img = augmented["image"]
        aug_bboxes = augmented["bboxes"]

        new_labels = []
        for bbox, cls in zip(aug_bboxes, class_labels):
            x_min, y_min, x_max, y_max = bbox
            x_center = (x_min + x_max) / 2
            y_center = (y_min + y_max) / 2
            bw = x_max - x_min
            bh = y_max - y_min
            if bw > 0 and bh > 0:
                new_labels.append([cls, x_center, y_center, bw, bh])

        if not new_labels:
            continue

        output_image_path = os.path.join(output_dir, "images", f"aug_{i}_{os.path.basename(image_path)}")
        output_label_path = os.path.join(output_dir, "labels", f"aug_{i}_{os.path.basename(label_path)}")

        cv2.imwrite(output_image_path, aug_img)
        save_yolo_labels(output_label_path, new_labels)

In [9]:
def process_image(image_file):
    image_path = os.path.join(r"F:\camera\Vehicle detection YOLOv8\dataset-vehicles\images\train", image_file) # images location 
    
    label_path = os.path.join(r"F:\camera\Vehicle detection YOLOv8\dataset-vehicles\labels\train", image_file.replace(".jpg", ".txt").replace(".png", ".txt"))
    augment_image(image_path, label_path, r"F:\camera\Vehicle detection YOLOv8\delete")


In [10]:
if __name__ == "__main__":
    os.makedirs(r"F:\camera\Vehicle detection YOLOv8\delete\images", exist_ok=True)
    os.makedirs(r"F:\camera\Vehicle detection YOLOv8\delete\labels", exist_ok=True)

    image_files = [f for f in os.listdir(r"F:\camera\Vehicle detection YOLOv8\dataset-vehicles\images\train") if f.endswith((".jpg", ".png"))]

    with ThreadPoolExecutor() as executor:
        list(tqdm(executor.map(process_image, image_files), total=len(image_files)))

    print("Fast Albumentations augmentation completed! 🚀")


  5%|███▋                                                                            | 54/1156 [00:00<00:17, 63.69it/s]

Skipping invalid bbox: [0.8972222222222224, 0.01796875, 0.9791666666666667, 0.0]


100%|██████████████████████████████████████████████████████████████████████████████| 1156/1156 [00:31<00:00, 36.39it/s]

Fast Albumentations augmentation completed! 🚀
